In [1]:
!pip install numpy

In [6]:
# perfomance analysis


import numpy as np

def check_binary_pattern_numpy(file_path, pattern="0101010101"):
    """
    Check if a file contains a specific binary pattern using NumPy.
    
    Args:
        file_path: Path to the file to check
        pattern: Binary pattern to search for (as a string of '0's and '1's)
    
    Returns:
        True if pattern is found, False otherwise
    """
    # Convert pattern string to array
    pattern_array = np.array([int(bit) for bit in pattern])
    
    # Read the file and convert to bit array
    with open(file_path, "rb") as f:
        data = np.frombuffer(f.read(), dtype=np.uint8)
    
    # Unpack bytes to bits
    bits = np.unpackbits(data)
    
    # Sliding window search
    pattern_len = len(pattern_array)
    for i in range(len(bits) - pattern_len + 1):
        if np.array_equal(bits[i:i+pattern_len], pattern_array):
            return True, i
    
    return False, -1

# Example usage
file_path = "rx_bits.bin"  # Replace with your file path
pattern = str("11100001010110101110100010010011")[-1]
found, position = check_binary_pattern_numpy(file_path, pattern)

if found:
    print(f"Pattern '{pattern}' found at bit position {position}")
    print(f"This corresponds to byte position {position // 8} and bit offset {position % 8}")
else:
    print(f"Pattern '{pattern}' not found in the file")


Pattern '1' found at bit position 95
This corresponds to byte position 11 and bit offset 7


In [7]:
# Performance Analysis for BPSK System

import numpy as np
import matplotlib.pyplot as plt
import os

def check_binary_pattern_numpy(file_path, pattern="0101010101"):
    """
    Check if a file contains a specific binary pattern using NumPy.
    
    Args:
        file_path: Path to the file to check
        pattern: Binary pattern to search for (as a string of '0's and '1's)
    
    Returns:
        True if pattern is found, False otherwise, and the position
    """
    # Convert pattern string to array
    pattern_array = np.array([int(bit) for bit in pattern])
    
    # Read the file and convert to bit array
    with open(file_path, "rb") as f:
        data = np.frombuffer(f.read(), dtype=np.uint8)
    
    # Unpack bytes to bits
    bits = np.unpackbits(data)
    
    # Sliding window search
    pattern_len = len(pattern_array)
    for i in range(len(bits) - pattern_len + 1):
        if np.array_equal(bits[i:i+pattern_len], pattern_array):
            return True, i
    
    return False, -1

def search_access_code(file_path, access_code_str="11100001010110101110100010010011"):
    """
    Search for an access code in a file, trying multiple bit/byte arrangements.
    
    Args:
        file_path: Path to the file to check
        access_code_str: Access code as a string of '0's and '1's
    """
    # Convert access code string to various formats for searching
    access_code_bits = np.array([int(bit) for bit in access_code_str])
    access_code_reversed = access_code_bits[::-1]  # Reversed bit order
    
    # Read the file
    try:
        with open(file_path, "rb") as f:
            data = f.read()
            
        print(f"File size: {len(data)} bytes")
        
        # Convert file to bits for bit-level search
        bits = np.unpackbits(np.frombuffer(data, dtype=np.uint8))
        
        # Search for normal bit pattern
        found = False
        for i in range(len(bits) - len(access_code_bits) + 1):
            if np.array_equal(bits[i:i+len(access_code_bits)], access_code_bits):
                print(f"Found access code at bit position {i} (bit-level search)")
                print(f"This corresponds to byte position {i // 8} and bit offset {i % 8}")
                found = True
                break
        
        if not found:
            print("Access code not found with normal bit order, trying reversed...")
            
        # Search for reversed bit pattern
        for i in range(len(bits) - len(access_code_reversed) + 1):
            if np.array_equal(bits[i:i+len(access_code_reversed)], access_code_reversed):
                print(f"Found access code at bit position {i} (reversed bit order)")
                print(f"This corresponds to byte position {i // 8} and bit offset {i % 8}")
                found = True
                break
        
        if not found:
            print("Access code not found in any format")
            
        # Try searching for partial matches
        print("\nSearching for partial matches...")
        best_match = 0
        best_pos = -1
        threshold = len(access_code_bits) * 0.8  # 80% match
        
        for i in range(len(bits) - len(access_code_bits) + 1):
            matches = np.sum(bits[i:i+len(access_code_bits)] == access_code_bits)
            if matches > best_match:
                best_match = matches
                best_pos = i
        
        match_percent = (best_match / len(access_code_bits)) * 100
        print(f"Best partial match: {best_match}/{len(access_code_bits)} bits ({match_percent:.1f}%) at position {best_pos}")
        
        if best_match > threshold:
            print("This is a strong partial match, suggesting the access code might be present with some errors")
            # Show the matched and mismatched bits
            matched = bits[best_pos:best_pos+len(access_code_bits)]
            print("Expected:", ''.join(str(b) for b in access_code_bits))
            print("Found:   ", ''.join(str(b) for b in matched))
            print("Matches: ", ''.join('|' if a == b else ' ' for a, b in zip(access_code_bits, matched)))
        
        return found, best_pos, best_match / len(access_code_bits)
        
    except Exception as e:
        print(f"Error processing file: {e}")
        return False, -1, 0

def analyze_ber_issue(tx_file="tx_bits.bin", rx_file="rx_bits.bin"):
    """
    Analyze BER calculation issues by comparing tx and rx files
    """
    # Check if files exist
    if not os.path.exists(tx_file):
        print(f"Error: {tx_file} not found")
        return
    if not os.path.exists(rx_file):
        print(f"Error: {rx_file} not found")
        return
    
    # Read files
    with open(tx_file, "rb") as f:
        tx_data = np.frombuffer(f.read(), dtype=np.uint8)
    with open(rx_file, "rb") as f:
        rx_data = np.frombuffer(f.read(), dtype=np.uint8)
    
    # Check if data is in packed or unpacked format
    tx_is_unpacked = np.all((tx_data == 0) | (tx_data == 1))
    rx_is_unpacked = np.all((rx_data == 0) | (rx_data == 1))
    
    print(f"TX data format: {'unpacked (1 bit per byte)' if tx_is_unpacked else 'packed (8 bits per byte)'}")
    print(f"RX data format: {'unpacked (1 bit per byte)' if rx_is_unpacked else 'packed (8 bits per byte)'}")
    
    # Convert to bits if needed
    tx_bits = tx_data if tx_is_unpacked else np.unpackbits(tx_data)
    rx_bits = rx_data if rx_is_unpacked else np.unpackbits(rx_data)
    
    print(f"TX bits length: {len(tx_bits)}")
    print(f"RX bits length: {len(rx_bits)}")
    
    # Calculate BER with different alignments
    min_ber = 1.0
    best_offset = 0
    
    # Try different alignments within a reasonable range
    max_offset = min(1000, max(len(tx_bits), len(rx_bits)))
    
    for offset in range(-max_offset, max_offset):
        if offset < 0:
            # RX is ahead of TX
            rx_slice = rx_bits[-offset:]
            tx_slice = tx_bits[:len(rx_slice)]
        else:
            # TX is ahead of RX
            tx_slice = tx_bits[offset:]
            rx_slice = rx_bits[:len(tx_slice)]
        
        # Ensure we have enough data to compare
        if len(tx_slice) < 100 or len(rx_slice) < 100:
            continue
        
        # Calculate BER
        min_len = min(len(tx_slice), len(rx_slice))
        errors = np.sum(tx_slice[:min_len] != rx_slice[:min_len])
        ber = errors / min_len
        
        # Also try inverted bits
        inverted_errors = np.sum(tx_slice[:min_len] != 1 - rx_slice[:min_len])
        inverted_ber = inverted_errors / min_len
        
        # Use the better of the two
        if inverted_ber < ber:
            ber = inverted_ber
            inverted = True
        else:
            inverted = False
        
        if ber < min_ber:
            min_ber = ber
            best_offset = offset
            best_inverted = inverted
    
    print(f"\nBest BER: {min_ber:.6f} at offset {best_offset} {'(inverted)' if best_inverted else ''}")
    
    # Show sample of aligned data
    if best_offset < 0:
        rx_aligned = rx_bits[-best_offset:]
        tx_aligned = tx_bits[:len(rx_aligned)]
    else:
        tx_aligned = tx_bits[best_offset:]
        rx_aligned = rx_bits[:len(tx_aligned)]
    
    min_len = min(len(tx_aligned), len(rx_aligned))
    if best_inverted:
        rx_aligned = 1 - rx_aligned
    
    print("\nSample of aligned data (first 50 bits):")
    print("TX:", ''.join(str(b) for b in tx_aligned[:50]))
    print("RX:", ''.join(str(b) for b in rx_aligned[:50]))
    print("Match:", ''.join('|' if a == b else ' ' for a, b in zip(tx_aligned[:50], rx_aligned[:50])))
    
    return min_ber, best_offset, best_inverted

# Search for access code in both files
print("Searching for access code in TX file:")
access_code = "11100001010110101110100010010011"
tx_found, tx_pos, tx_match = search_access_code("tx_bits.bin", access_code)

print("\nSearching for access code in RX file:")
rx_found, rx_pos, rx_match = search_access_code("rx_bits.bin", access_code)

# Analyze BER issues
print("\nAnalyzing BER calculation issues:")
min_ber, best_offset, best_inverted = analyze_ber_issue()

# Recommendations based on findings
print("\n=== RECOMMENDATIONS ===")
if not tx_found and not rx_found:
    print("1. The access code was not found in either file. This suggests:")
    print("   - The access code might be in a different format than expected")
    print("   - The correlate block might have removed the access code from both files")
    print("   - Try checking with a different access code format (e.g., byte-packed)")
elif tx_found and not rx_found:
    print("1. The access code was found in TX but not in RX. This suggests:")
    print("   - The correlate block might have removed the access code from RX")
    print("   - You should modify your BER calculator to account for this difference")
    print("   - Try removing the access code from TX before BER calculation")
elif not tx_found and rx_found:
    print("1. The access code was found in RX but not in TX. This is unusual and suggests:")
    print("   - The files might be from different transmissions")
    print("   - There might be format differences between the files")
    print("   - Check that you're using the correct files")

if min_ber > 0.4:
    print("2. The BER is very high (> 40%) even with alignment attempts. This suggests:")
    print("   - There might be a fundamental format mismatch between TX and RX")
    print("   - The files might be from different transmissions")
    print("   - Try generating new reference data with the same processing chain")
else:
    print(f"2. Found a reasonable BER of {min_ber:.6f} at offset {best_offset}. To fix your BER calculation:")
    print(f"   - {'Invert the bits and ' if best_inverted else ''}Adjust the offset by {best_offset} bits")
    print("   - Modify your BER calculator to use this alignment")

print("\n3. For your BER calculator:")
if best_inverted:
    print("   - Add code to invert the bits before comparison")
print(f"   - Add code to adjust the offset by {best_offset} bits")
print("   - Consider using a sliding window approach to find the best alignment for each packet")

Searching for access code in TX file:
File size: 75744 bytes
Found access code at bit position 0 (bit-level search)
This corresponds to byte position 0 and bit offset 0

Searching for partial matches...
Best partial match: 32/32 bits (100.0%) at position 0
This is a strong partial match, suggesting the access code might be present with some errors
Expected: 11100001010110101110100010010011
Found:    11100001010110101110100010010011
Matches:  ||||||||||||||||||||||||||||||||

Searching for access code in RX file:
File size: 11366 bytes
Access code not found with normal bit order, trying reversed...
Access code not found in any format

Searching for partial matches...
Best partial match: 19/32 bits (59.4%) at position 118

Analyzing BER calculation issues:
TX data format: packed (8 bits per byte)
RX data format: unpacked (1 bit per byte)
TX bits length: 605952
RX bits length: 11366

Best BER: 0.005916 at offset -40 

Sample of aligned data (first 50 bits):
TX: 111000010101101011101000100